<a href="https://colab.research.google.com/github/Sid200026/Misc-Programs/blob/master/CUDA/Introduction_to_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [18]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


# Hello World

In [9]:
# The __global__ specifier indicates a function that runs on device (GPU)
# Such function can be called through host code

# CPU -> host
# GPU -> device

In [29]:
%%cu
#include<stdio.h>
__global__ void cuda_hello() {
    printf("Hello world from CUDA"); // Printing doesn't produce an output to the kernel
}

main() {
    cuda_hello<<<1,1>>>();
}

# Vector Addition

In [ ]:
"""
CPU and GPUs are separate entities.  Both have their own memory space. 
CPU cannot directly access GPU memory, and vice versa. 
In CUDA terminology, CPU memory is called host memory and GPU memory is called device memory. 
Pointers to CPU and GPU memory are called host pointer and device pointer, respectively.

For data to be accessible by GPU, it must be presented in the device memory. 
CUDA provides APIs for allocating device memory and data transfer between host and device memory. 
Following is the common workflow of CUDA programs.

Allocate host memory and initialized host data
Allocate device memory
Transfer input data from host to device memory
Execute kernels
Transfer output from device memory to host

cudaMalloc(void **devPtr, size_t count);
cudaFree(void *devPtr);
cudaMemcpy(void *dst, void *src, size_t count, cudaMemcpyKind kind)
"""

In [54]:
%%cu
#include<stdio.h>
#define N 10

__global__ void cuda_add(float* c, float* b, float* a) {
    for(int i=0; i<N; i++) {
        c[i] = *(a+i) + *(b+i);
    }
}

int main() {
    float *a, *b, *c;
    // Host Memory
    a = (float*)malloc(sizeof(float)*N);
    b = (float*)malloc(sizeof(float)*N);
    c = (float*)malloc(sizeof(float)*N);
    for(int i=0; i<N; i++) {
        a[i] = 1.2f*i;
        b[i] = 2.3f*i/2;
    }
    // Device Memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, sizeof(float)*N);
    cudaMalloc((void**)&d_b, sizeof(float)*N);
    cudaMalloc((void**)&d_c, sizeof(float)*N);
    // Host -> Device
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);
    cuda_add<<<1,1>>>(d_c, d_a, d_b);
    cudaMemcpy(c, d_c, sizeof(float) * N, cudaMemcpyDeviceToHost);
    for(int i=0; i<5; i++) {
        printf("%f\n", c[i]);
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(a);
    free(b);
    free(c);
    
}

0.000000
2.350000
4.700000
7.050000
9.400000



# Parallelism

In [ ]:
"""
The above two programs uses 1 GPU thread ( <<<1,1>>> )

CUDA organizes threads into a group called "thread block".
Kernel can launch multiple thread blocks, organized into a "grid" structure.

<<< M , T >>>
Which indicate that a kernel launches with a grid of M thread blocks. 
Each thread block has T parallel threads.

blockIdx.x contains the index of the block with in the grid
gridDim.x contains the size of the grid
threadIdx.x contains the index of the thread within the block
blockDim.x contains the size of thread block (number of threads in the thread block).
"""

### Vector Addition

In [56]:
%%cu
#include<stdio.h>
#define N 10

__global__ void cuda_add(float* c, float* b, float* a) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if(tid < N)
      c[tid] = *(a+tid) + *(b+tid);
}

int main() {
    float *a, *b, *c;
    // Host Memory
    a = (float*)malloc(sizeof(float)*N);
    b = (float*)malloc(sizeof(float)*N);
    c = (float*)malloc(sizeof(float)*N);
    for(int i=0; i<N; i++) {
        a[i] = 1.2f*i;
        b[i] = 2.3f*i/2;
    }
    // Device Memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, sizeof(float)*N);
    cudaMalloc((void**)&d_b, sizeof(float)*N);
    cudaMalloc((void**)&d_c, sizeof(float)*N);
    // Host -> Device
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);
    int block_size = 256;
    int grid_size = ((N + block_size) / block_size);
    cuda_add<<<grid_size,block_size>>>(d_c, d_a, d_b);
    cudaMemcpy(c, d_c, sizeof(float) * N, cudaMemcpyDeviceToHost);
    for(int i=0; i<5; i++) {
        printf("%f\n", c[i]);
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(a);
    free(b);
    free(c); 
}

0.000000
2.350000
4.700000
7.050000
9.400000



# Practice Problems

### Multiply Vector By a constant

In [69]:
%%cu
#include<stdio.h>
#define N 15
#define constant 8

__global__ void cuda_multiply(int *vector) {
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id < N) {
      vector[thread_id] = vector[thread_id]*constant;        
    }
}

__global__ void cuda_random_assign(int *vector) {
    int thread_num = threadIdx.x;
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id < N) {
      vector[thread_id] = thread_num;        
    }
}

int main() {
    
    int block_size = 4;
    int grid_size = (block_size+N)/block_size;

    int *vector;
    vector = (int*)malloc(sizeof(int) * N);

    int *d_vector_assign;
    cudaMalloc((void**)&d_vector_assign, sizeof(int)*N);
    cudaMemcpy(d_vector_assign, vector, sizeof(int) * N, cudaMemcpyHostToDevice);

    cuda_random_assign<<<grid_size, block_size>>>(d_vector_assign);
    cudaMemcpy(vector, d_vector_assign, sizeof(int) * N, cudaMemcpyDeviceToHost);
    cudaFree(d_vector_assign);

    int *d_vector;
    cudaMalloc((void**)&d_vector, sizeof(int)*N);
    cudaMemcpy(d_vector, vector, sizeof(int) * N, cudaMemcpyHostToDevice);

    cuda_multiply<<<grid_size, block_size>>>(d_vector);
    cudaMemcpy(vector, d_vector, sizeof(int) * N, cudaMemcpyDeviceToHost);
    cudaFree(d_vector);
    for(int i=0; i<N; i++) {
        printf("%d ", vector[i]);
    }
    free(vector);
}

0 8 16 24 0 8 16 24 0 8 16 24 0 8 16 


### Square of values of a matrix

In [139]:
%%cu
#include<stdio.h>
#define N 6

// Flattens the 2D Array into a 1D following Row-Major

__global__ void cuda_square(int *matrix) {
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    if(row < N*N) {
      matrix[row] = matrix[row] * matrix[row] ;
    }
}

int main() {
    int block_size = N;
    int grid_size = N;
    int *matrix;
    matrix = (int*)malloc(sizeof(int)*N*N);
    for(int i=0; i<N; i++) {
        for(int j=0; j<N; j++) {
            int num = j*i+j;
            matrix[i*N+j] = num;
        }
    }
    int *d_matrix;
    cudaMalloc((void**)&d_matrix, sizeof(int)*N*N);
    cudaMemcpy(d_matrix, matrix, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cuda_square<<<grid_size, block_size>>>(d_matrix);
    cudaMemcpy(matrix, d_matrix, sizeof(int) * N * N, cudaMemcpyDeviceToHost);
    cudaFree(d_matrix);
    for(int i=0; i<N; i++) {
        for(int j=0; j<N; j++) {
          printf("%d ", matrix[i*N + j]);   
        }
        printf("\n");
    }    
    free(matrix);
}

0 1 4 9 16 25 
0 4 16 36 64 100 
0 9 36 81 144 225 
0 16 64 144 256 400 
0 25 100 225 400 625 
0 36 144 324 576 900 



### Multiples of all elements in an array

In [130]:
%%cu
#include<stdio.h>
#define N 6
#define M 3

__global__ void cuda_square(int *vector, int *matrix) {
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    for(int i=0; i<M; i++) {
        matrix[row*M+i] = vector[row] * (i+1);
    }
}

int main() {
    int block_size = N/2;
    int grid_size = (block_size+N)/block_size;
    int *vector, *matrix;
    vector = (int*)malloc(sizeof(int)*N);
    matrix = (int*)malloc(sizeof(int)*N*M);
    for(int i=0; i<N; i++) {
        vector[i] = i+1;
    }
    int *d_vector, *d_matrix;
    cudaMalloc((void**)&d_vector, sizeof(int)*N);
    cudaMemcpy(d_vector, vector, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMalloc((void**)&d_matrix, sizeof(int)*N*M);
    cuda_square<<<grid_size, block_size>>>(d_vector, d_matrix);
    cudaMemcpy(matrix, d_matrix, sizeof(int) * N * M, cudaMemcpyDeviceToHost);
    cudaFree(d_matrix);
    for(int i=0; i<N; i++) {
        for(int j=0; j<M; j++) {
          printf("%d ", matrix[i*M + j]);   
        }
        printf("\n");
    }    
    free(matrix);
}

1 2 3 
2 4 6 
3 6 9 
4 8 12 
5 10 15 
6 12 18 



### Rank of a matrix

In [185]:
%%cu
#include<stdio.h>
#define N 4
// N rows, N columns

__global__ void cuda_transpose(int *matrix, int *rank) {
    bool *flag = (bool*)malloc(sizeof(bool) * N);
    for(int i=0; i<N; i++) {
        flag[i] = false;
    }
    for(int j=0; j<N; j++) {
        if(flag[j])
            continue;
        for(int i=j+1; i<N; i++) {
            if(flag[i])
                continue;
            int div = -1;
            for(int k=0; k<N; k++) {
              int div_2 = matrix[i+k*N] / matrix[j+k*N];
              if(matrix[i+k*N] % matrix[j+k*N] != 0) {
                  div = -1;
                  break;
              }
              if(div == -1)
                  div = div_2;
              if(div != div_2) {
                  div = -1;
                  break;
              }
            }
            if(div != -1) {
                flag[i] = true;
                rank[0] -= 1;
            }
        }
    }
}

int main() {
    int block_size = 1;
    int grid_size = 1;
    int *matrix = (int*)malloc(sizeof(int) * N * N);
    int *rank = (int*)malloc(sizeof(int));
    rank[0] = N;
    for(int i=0; i<N*N; i++) {
        matrix[i] = i+2;
    }
    for(int i=0; i<N; i++) {
        for(int j=0; j<N; j++) {
            printf("%d ", matrix[i*N+j]);
        }
        printf("\n");
    }
    int *d_matrix, *d_rank;
    cudaMalloc((void**)&d_matrix, sizeof(int)*N*N);
    cudaMemcpy(d_matrix, matrix, sizeof(int) * N*N, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&d_rank, sizeof(int));
    cudaMemcpy(d_rank, new int(N), sizeof(int), cudaMemcpyHostToDevice);

    cuda_transpose<<<grid_size, block_size>>>(d_matrix, d_rank);
    cudaMemcpy(rank, d_rank, sizeof(int), cudaMemcpyDeviceToHost);
    printf("%d", *(rank));
}

2 3 4 5 
6 7 8 9 
10 11 12 13 
14 15 16 17 
4


### Transpose of a matrix

In [140]:
%%cu
#include<stdio.h>
#define N 4
#define M 4
// N rows, M columns

__global__ void cuda_transpose(int *matrix, int *transpose) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int row = tid / N;
    int col = tid-row*N;
    if(row*N + col < (N*M)) {
        transpose[col*M + row] = matrix[row*N + col];
    }
}

int main() {
    int block_size = N/2;
    int grid_size = (block_size+N*M)/block_size;
    int *matrix = (int*)malloc(sizeof(int) * M * N);
    for(int i=0; i<N*M; i++) {
        matrix[i] = i+1;
    }
    int *d_matrix, *d_transpose;
    cudaMalloc((void**)&d_matrix, sizeof(int)*N*M);
    cudaMemcpy(d_matrix, matrix, sizeof(int) * N*M, cudaMemcpyHostToDevice);
    cudaMalloc((void**)&d_transpose, sizeof(int)*N*M);
    cuda_transpose<<<grid_size, block_size>>>(d_matrix, d_transpose);
    cudaMemcpy(matrix, d_transpose, sizeof(int) * N * M, cudaMemcpyDeviceToHost);
    for(int i=0; i<M; i++) {
        for(int j=0; j<N; j++) {
            printf("%d ", matrix[i*M+j]);
        }
        printf("\n");
    }
}

1 5 9 13 
2 6 10 14 
3 7 11 15 
4 8 12 16 



### Reverse an array

In [142]:
%%cu
#include<stdio.h>
#define N 10

__global__ void cuda_reverse(int* vector) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if(tid<=N/2) {
        int other_index = N-tid-1;
        int temp = vector[other_index];
        vector[other_index] = vector[tid];
        vector[tid] = temp;
    }
}

int main() {
    int block_size = N/2;
    int grid_size = (block_size+N)/block_size;
    int *vector = (int*)malloc(sizeof(int) * N);
    for(int i=0; i<N; i++) {
        vector[i] = i+1;
    }
    int *d_vector;
    cudaMalloc((void**)&d_vector, sizeof(int)*N);
    cudaMemcpy(d_vector, vector, sizeof(int)*N, cudaMemcpyHostToDevice);
    cuda_reverse<<<grid_size, block_size>>>(d_vector);
    cudaMemcpy(vector, d_vector, sizeof(int)*N, cudaMemcpyDeviceToHost);
    for(int j=0; j<N; j++) {
        printf("%d ", vector[j]);
    }
}

10 9 8 7 6 5 4 3 2 1 


### Multiply 3 vectors

In [147]:
%%cu
#include<stdio.h>
#define N 15

__global__ void cuda_multiply(int *vector1, int *vector2, int *vector3) {
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_id < N) {
      vector1[thread_id] *= vector2[thread_id]*vector3[thread_id];        
    }
}

int main() {
    
    int grid_size = 3;
    int block_size = (N/grid_size)+1;

    int *vector1, *vector2, *vector3;
    vector1 = (int*)malloc(sizeof(int) * N);
    vector2 = (int*)malloc(sizeof(int) * N);
    vector3 = (int*)malloc(sizeof(int) * N);

    // Random value generator
    for(int i=0; i<N; i++) {
        vector1[i] = 4*(i+1);
        vector2[i] = vector1[i] +1;
        vector3[i] = 2*i;
    }

    int *d_vector1, *d_vector2, *d_vector3;

    cudaMalloc((void**)&d_vector1, sizeof(int)*N);
    cudaMemcpy(d_vector1, vector1, sizeof(int) * N, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&d_vector2, sizeof(int)*N);
    cudaMemcpy(d_vector2, vector2, sizeof(int) * N, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&d_vector3, sizeof(int)*N);
    cudaMemcpy(d_vector3, vector3, sizeof(int) * N, cudaMemcpyHostToDevice);

    cuda_multiply<<<grid_size, block_size>>>(d_vector1, d_vector2, d_vector3);
    cudaMemcpy(vector1, d_vector1, sizeof(int) * N, cudaMemcpyDeviceToHost);
    for(int i=0; i<N; i++) {
        printf("%d ", vector1[i]);
    }
    free(vector1);
    free(vector2);
    free(vector3);
}

0 144 624 1632 3360 6000 9744 14784 21312 29520 39600 51744 66144 82992 102480 
